In [ ]:
import pandas as pd
import numpy as np
import matplotlib.mlab as ml
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable

import pygmt as gmt
import xarray as xr

import rasterio
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling

from shapely.geometry import box
import geopandas as gpd
from fiona.crs import from_epsg
import pycrs

from scripts import morphoGrid as morph
from scipy.spatial import cKDTree

# display plots in SVG format
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

# Get exposed shelf percentage

### Intermittent flooding

+ -415 (584) -400 (600) -393 (607)
+ -337 (663) -325 (675) -304 (696) 
+ -246 (754) -237 (763) -230 (770)
+ -220 (780) -213 (787) -190 (810)
+ -107 (893) -81 (919)  -70 (930)

### Full flooding

+ -132 (868) -124 (876) -112 (888)
+ -16 (984) 0 (1000)

# Read badlands output file

In [ ]:
out_tif_utm2 = r'data/elevation/clip_utm1.tif'
clipped = rasterio.open(out_tif_utm2)

In [ ]:
step = 430
dx = 5000.
folder = 'lowsub/h5'

badland_topo = morph.morphoGrid(folder=folder, bbox = None, dx=dx)
badland_topo.loadHDF5(timestep=step)
height = badland_topo.z.shape[0]
width = badland_topo.z.shape[1]
minx = badland_topo.x.min()
maxy = badland_topo.y.max()
sealevel = badland_topo.sl

# Build output as geotiff

In [ ]:
bad_z = r'elev'+str(step)+'.tif'
bad_fa = r'fa'+str(step)+'.tif'
bad_ed = r'ed'+str(step)+'.tif'

bad_z_png = r'out_z'+str(step)+'.png'
bad_ed_png = r'out_e'+str(step)+'.png'

In [ ]:
profile = {
        'driver': clipped.driver,
        'dtype': rasterio.float32,
        'nodata': None,
        'width': width, 
        'height': height,
        'count': 1,
        'crs': clipped.crs,
        'transform': rasterio.Affine(dx, 0.0, minx, 0.0, -dx, maxy),
        'tiled': False, 
        'interleave': 'band'
    }

data = np.flipud(badland_topo.z-sealevel)
band_number = 1

profile.update(dtype=data.dtype)

with rasterio.open(bad_z, 'w', **profile) as dst:
    dst.write(data, band_number)

In [ ]:
data2 = np.flipud(np.log10(badland_topo.discharge))
band_number = 1

profile.update(dtype=data2.dtype)

with rasterio.open(bad_fa, 'w', **profile) as dst:
    dst.write(data2, band_number)

In [ ]:
data3 = np.flipud(badland_topo.cumchange)
band_number = 1

profile.update(dtype=data3.dtype)

with rasterio.open(bad_ed, 'w', **profile) as dst:
    dst.write(data3, band_number)

# Plot badlands outputs

In [ ]:
fig = gmt.Figure()

fig.grdimage(
    grid=bad_z,
    projection="x1:60000000",
#     frame="f",  # add minor tick labels only
    cmap="geo",
    Q=True,
    I=True,
    n='b'
)

gmt.makecpt(cmap="devon", 
            series=[7,9],
            I=True,
           )

fig.grdimage(
    grid=bad_fa,
    projection="x1:60000000",
#     frame="f",  # add minor tick labels only
    Q=True,
    C=True,
    t=90,
    n='b'   
)

fig.grdcontour(
    grid=bad_z,
    interval=10000.,
    pen="0.25p"
    )

# Create a colorbar designed for seismic tomography- roma
# fig.colorbar(
#     cmap="geo",
#     frame=["+Lelevation", "xaf", "y+lm"],
#     scale=1,
# )
fig.savefig(fname=bad_z_png,dpi=1000, crop=True)
fig.show(dpi=1000, width=800)

In [ ]:
fig = gmt.Figure()

# fig.grdimage(
#     grid=bad_z,
#     projection="x1:60000000",
# #     frame="f",  # add minor tick labels only
#     cmap="geo",
#     Q=True,
#     I=True,
#     n='b'
# )

gmt.makecpt(cmap="polar", 
            series=[-50,50],
            I=False,
           )

fig.grdimage(
    grid=bad_ed,
    projection="x1:60000000",
#     frame="f",  # add minor tick labels only
    Q=True,
    C=True,
#     t=90,
    n='b'   
)

gmt.makecpt(cmap="devon", 
            series=[7,9],
            I=True,
           ) 

fig.grdimage(
    grid=bad_fa,
    projection="x1:60000000",
#     frame="f",  # add minor tick labels only
    Q=True,
    C=True,
    t=70,
    n='b'   
)


fig.grdcontour(
    grid=bad_z,
    interval=10000.,
    pen="0.25p"
    )

# Create a colorbar designed for seismic tomography- roma
# fig.colorbar(
#     cmap="geo",
#     frame=["+Lelevation", "xaf", "y+lm"],
#     scale=1,
# )
fig.savefig(fname=bad_ed_png,dpi=1000, crop=True)
fig.show(dpi=1000, width=800)

STOP HERE

In [ ]:
fig = gmt.Figure()

#gmt.makecpt(cmap="oleron", series=[-4500, 4500])

fig.grdimage(
    grid=bad_z,
#     region=[-2700000, 2800000, -2200000, 2300000],
    projection="x1:60000000",
#     frame="f",  # add minor tick labels only
    cmap="geo",
    Q=True,
    I=True,
    n='b'
)
fig.grdcontour(
    grid=bad_z,
    interval=10000.,
    pen="0.25p"
#     annotation="1+f6p",
#     limit="-8000/0",
#     pen="a0.15p"
    )
fig.show(dpi=720, width=800)

In [ ]:
STOP

In [ ]:
dst_crs = 'EPSG:4326'
bad_lonlat = r'data/elevation/clip_utm1.tif'

In [ ]:
transform

In [ ]:
width,width0,height,height0

In [ ]:
with rasterio.open(bad_tif) as src:
    transform0, width0, height0 = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    dll = transform[0]
    transform1 = rasterio.Affine(transform0[0], 0.0, 
                                 -transform0[2]-100., 
                                 0.0, transform0[4], 
                                 transform0[5])
    kwargs.update({
        'crs': dst_crs,2
        'transform': transform1,
        'width': width0,
        'height': height
    })

    with rasterio.open(bad_lonlat, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)
            
clipped_utm = rasterio.open(bad_lonlat)
fig, ax = plt.subplots(1,1, figsize=(12,7))
show(clipped_utm, ax=ax, cmap='gist_earth', title='Elevation')
plt.show()

In [ ]:
minlx, minly = 160., -11.00
maxlx, maxly = 200., 20.

In [ ]:
bbox2 = box(minlx, minly, maxlx, maxly)
geo2 = gpd.GeoDataFrame({'geometry': bbox2}, index=[0], crs=from_epsg(4326))
geo2 = geo2.to_crs(crs=clipped_utm.crs.data)

coords2 = getFeatures(geo2)
#print(coords2)

out_utm2, out_transform2 = mask(clipped_utm, shapes=coords2, crop=True)
out_meta2 = clipped_utm.meta.copy()
epsg_code2 = int(clipped_utm.crs.data['init'][5:])
# print(out_meta2)
# print(epsg_code2)

out_meta2.update({"driver": "GTiff",
                "height": out_utm2.shape[1],          
                 "width": out_utm2.shape[2],
                 "transform": out_transform2,
                 "crs": pycrs.parse.from_epsg_code(epsg_code2).to_proj4()}                 
               )

with rasterio.open(out_tif_utm2, "w", **out_meta2) as dest:
    dest.write(out_utm2)
    
clipped = rasterio.open(out_tif_utm2)
fig, ax = plt.subplots(1,1, figsize=(12,7))
show(clipped, ax=ax, cmap='gist_earth', title='Elevation')
plt.show()

In [ ]:
clipped_utm.profile

In [ ]:
{'dtype': 'int16', 'tiled': False, 'count': 1, 'crs': CRS.from_epsg(4326), 'width': 3601, 'transform': Affine(0.016666666666666666, 0.0, 79.99166666666667,
       0.0, -0.016666666666666663, 20.008333333333354), 'interleave': 'band', 'nodata': None, 'height': 2401, 'driver': 'GTiff'}


In [ ]:
clipped_utm.profile

In [ ]:
# Register GDAL format drivers and configuration options with a
# context manager.
with rasterio.Env():

    # Write an array as a raster band to a new 8-bit file. For
    # the new file's profile, we start with the profile of the source
    profile = clipped.profile

    # And then change the band count to 1, set the
    # dtype to uint8, and specify LZW compression.
    profile.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open('example.tif', 'w', **profile) as dst:
        dst.write(array.astype(rasterio.uint8), 1)

# At the end of the ``with rasterio.Env()`` block, context
# manager exits

In [ ]:
minux, minuy = -1800000., -1400000.
maxux, maxuy = 3000000., 2200000.

bbox2 = box(minux, minuy, maxux, maxuy)
geo2 = gpd.GeoDataFrame({'geometry': bbox2}, index=[0], crs=from_epsg(32649))
geo2 = geo2.to_crs(crs=clipped_utm.crs.data)

coords2 = getFeatures(geo2)
print(coords2)

out_utm2, out_transform2 = mask(clipped_utm, shapes=coords2, crop=True)
out_meta2 = clipped_utm.meta.copy()
epsg_code2 = int(clipped_utm.crs.data['init'][5:])
print(out_meta2)
print(epsg_code2)

out_meta2.update({"driver": "GTiff",
                "height": out_utm2.shape[1],          
                 "width": out_utm2.shape[2],
                 "transform": out_transform2,
                 "crs": pycrs.parse.from_epsg_code(epsg_code2).to_proj4()}                 
               )

with rasterio.open(out_tif_utm2, "w", **out_meta2) as dest:
    dest.write(out_utm2)
    
clipped = rasterio.open(out_tif_utm2)
fig, ax = pyplot.subplots(1,1, figsize=(12,7))
show(clipped, ax=ax, cmap='gist_earth', title='Elevation')
pyplot.show()

In [ ]:
badland_topo = morph.morphoGrid(folder='lowsub/h5', bbox = None, dx=5000)
badland_topo.loadHDF5(timestep=1000)

In [ ]:
regX = badland_topo.x.flatten()
regY = badland_topo.y.flatten()
regXY = np.vstack([regX, regY]).T

In [ ]:
plt.figure(figsize=(10, 8))
ax = plt.gca()
im = ax.imshow(badland_topo.z, cmap ='seismic', vmin = -1000, vmax = 1000, 
                    interpolation ='nearest', origin ='lower') 
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)

plt.colorbar(im, cax=cax)
plt.show()